In [1]:
import numpy as np 
import pandas as pd 
import torch
from data_processing import generate_vocab, process_data, create_dataloaders
from model import get_pretrained_emb, EncoderRNN, DecoderRNN, DecoderAttnRNN, EncoderDecoder, EncoderDecoderAttn
from train_eval import train_and_eval, count_parameters, summarize_results, plot_single_learning_curve, load_experiment_log
import pickle as pkl 
from datetime import datetime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# params required for generating data loaders 

SRC_LANG = 'vi'
TARG_LANG = 'en'

SRC_MAX_SENTENCE_LEN = 10
TARG_MAX_SENTENCE_LEN = 10
SRC_VOCAB_SIZE = 30000 
TARG_VOCAB_SIZE = 30000 

BATCH_SIZE = 64 

In [3]:
# takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [4]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, filter_long=False)
data_minibatch = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=BATCH_SIZE, filter_long=False) 
data_minitrain = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=1000, filter_long=False)

In [5]:
# create dataloaders 
loaders_full = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minibatch = create_dataloaders(data_minibatch, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minitrain = create_dataloaders(data_minitrain, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [6]:
# model architecture params 
NETWORK_TYPE = 'rnn'
RNN_CELL_TYPE = 'gru'
NUM_LAYERS = 2 
ENC_HIDDEN_DIM = 512
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM 
TEACHER_FORCING_RATIO = 1
CLIP_GRAD_MAX_NORM = 1
ENC_DROPOUT = 0.2 
DEC_DROPOUT = 0.2  
ATTENTION_TYPE = 'additive'

# training params  
NUM_EPOCHS = 15 
LR = 0.0001 
OPTIMIZER = 'Adam'
LAZY_TRAIN = False

# name the model and experiment 
if NETWORK_TYPE == 'rnn': 
    EXPERIMENT_NAME = '{}-rnn-{}-attn'.format(SRC_LANG, ATTENTION_TYPE)
elif NETWORK_TYPE == 'cnn': 
    EXPERIMENT_NAME = '{}-cnn'.format(SRC_LANG)
MODEL_NAME = '{}-{}'.format(EXPERIMENT_NAME, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [7]:
# store as dict to save to results later 
params = {'experiment_name': EXPERIMENT_NAME,'model_name': MODEL_NAME, 'src_lang': SRC_LANG, 'targ_lang': TARG_LANG, 
          'rnn_cell_type': RNN_CELL_TYPE, 'src_max_sentence_len': SRC_MAX_SENTENCE_LEN, 
          'targ_max_sentence_len': TARG_MAX_SENTENCE_LEN, 'src_vocab_size': SRC_VOCAB_SIZE, 
          'targ_vocab_size': TARG_VOCAB_SIZE, 'num_layers': NUM_LAYERS, 'enc_hidden_dim': ENC_HIDDEN_DIM, 
          'dec_hidden_dim': DEC_HIDDEN_DIM, 'teacher_forcing_ratio': TEACHER_FORCING_RATIO, 
          'clip_grad_max_norm': CLIP_GRAD_MAX_NORM, 'enc_dropout': ENC_DROPOUT, 'dec_dropout': DEC_DROPOUT, 
          'attention_type': ATTENTION_TYPE, 'batch_size': BATCH_SIZE, 'num_epochs': NUM_EPOCHS, 
          'learning_rate': LR, 'optimizer': OPTIMIZER, 'lazy_train': LAZY_TRAIN} 

In [8]:
# define model 

encoder = EncoderRNN(rnn_cell_type=RNN_CELL_TYPE, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                     src_max_sentence_len=SRC_MAX_SENTENCE_LEN, enc_dropout=ENC_DROPOUT, 
                     pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))

if ATTENTION_TYPE == 'without': 
    # without attention 
    decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS,
                         targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                         pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], 
                                                                vocab[TARG_LANG]['token2id']))
    model = EncoderDecoder(encoder, decoder, vocab[TARG_LANG]['token2id']).to(device)
    
else: 
    # with attention 
    decoder = DecoderAttnRNN(rnn_cell_type=RNN_CELL_TYPE, dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, 
                             num_layers=NUM_LAYERS, targ_vocab_size=TARG_VOCAB_SIZE, 
                             src_max_sentence_len=SRC_MAX_SENTENCE_LEN, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                             dec_dropout=DEC_DROPOUT, attention_type=ATTENTION_TYPE,
                             pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], 
                                                                    vocab[TARG_LANG]['token2id']))
    model = EncoderDecoderAttn(encoder, decoder, vocab[TARG_LANG]['token2id']).to(device) 

In [ ]:
model, results = train_and_eval(
    model=model, loaders_full=loaders_full, loaders_minibatch=loaders_minibatch, loaders_minitrain=loaders_minitrain, 
    params=params, vocab=vocab, print_intermediate=100, save_checkpoint=True, save_to_log=True, 
    lazy_eval=True, print_attn=True, inspect_samples=1)

Epoch: 0.00, Train Loss: 0.00, Val Loss: 10.29, Train BLEU: 0.00, Val BLEU: 0.00, Minutes Elapsed: 0.13
Sampling from val predictions...
Source: chúng_ta những nạn_nhân cần đến tất_cả mọi người . <EOS>
Reference: we victims need everyone . <EOS> <PAD> <PAD> <PAD>
Model: <SOS> caption caption caption caption elitist elitist elitist freely freely
Attention Weights: tensor([[0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000, 0.1001, 0.1001, 0.1007, 0.1006, 0.1005,
         0.1027],
        [0.0986, 0.0983, 0.0985, 0.1000,

Epoch: 0.10, Train Loss: 0.00, Val Loss: 6.21, Train BLEU: 0.00, Val BLEU: 1.30, Minutes Elapsed: 5.19
Sampling from val predictions...
Source: các gia_đình hàng_xóm nghe kể về ý_tưởng này . <EOS>
Reference: and my neighboring homes heard about this idea .
Model: <SOS> and &apos;s the the . . . . .
Attention Weights: tensor([[8.3157e-06, 4.1053e-03, 8.9195e-02, 2.5529e-01, 2.5606e-01, 2.0147e-01,
         1.3048e-01, 5.1371e-02, 9.3558e-03, 2.6709e-03],
        [8.7240e-05, 1.3469e-02, 1.0965e-01, 2.1270e-01, 2.0989e-01, 1.7924e-01,
         1.3910e-01, 8.2685e-02, 3.1731e-02, 2.1444e-02],
        [1.4440e-04, 5.4484e-03, 2.8936e-02, 5.1302e-02, 5.0970e-02, 4.5558e-02,
         4.0251e-02, 3.4335e-02, 3.8145e-02, 7.0491e-01],
        [1.2625e-05, 3.3452e-04, 1.8714e-03, 3.4943e-03, 3.4164e-03, 3.0291e-03,
         2.7818e-03, 2.7626e-03, 5.7113e-03, 9.7659e-01],
        [4.1132e-06, 1.0739e-04, 6.6137e-04, 1.2866e-03, 1.2309e-03, 1.0732e-03,
         9.9927e-04, 1.0725e-03, 3.0761e-03,

Epoch: 0.14, Train Loss: 0.00, Val Loss: 6.15, Train BLEU: 0.00, Val BLEU: 1.36, Minutes Elapsed: 7.69
Sampling from val predictions...
Source: bây_giờ tôi muốn giới_thiệu các bạn với những người em_trai
Reference: now i &apos;d like to introduce you to my
Model: <SOS> and , , , , , , , ,
Attention Weights: tensor([[1.7282e-06, 3.5568e-05, 1.3156e-03, 3.7219e-02, 1.7338e-01, 3.2576e-01,
         3.0054e-01, 1.5262e-01, 9.1044e-03, 2.7114e-05],
        [1.7126e-05, 3.0989e-04, 6.4648e-03, 7.2837e-02, 1.9150e-01, 2.8140e-01,
         2.6474e-01, 1.6437e-01, 1.8301e-02, 6.2360e-05],
        [1.4059e-04, 1.7037e-03, 1.8358e-02, 9.8321e-02, 1.8429e-01, 2.3224e-01,
         2.2533e-01, 1.7845e-01, 5.9534e-02, 1.6301e-03],
        [5.3611e-04, 3.6287e-03, 2.3545e-02, 9.4603e-02, 1.6717e-01, 2.0627e-01,
         2.0373e-01, 1.7875e-01, 1.0259e-01, 1.9182e-02],
        [7.2082e-04, 3.8620e-03, 2.1698e-02, 8.5845e-02, 1.5745e-01, 1.9778e-01,
         1.9591e-01, 1.7526e-01, 1.1587e-01, 4.5603e-0

Epoch: 0.19, Train Loss: 0.00, Val Loss: 6.14, Train BLEU: 0.00, Val BLEU: 1.44, Minutes Elapsed: 10.19
Sampling from val predictions...
Source: cô cũng đã có bản_sao của bức ảnh . <EOS>
Reference: she also had <UNK> . <EOS> <PAD> <PAD> <PAD>
Model: <SOS> and &apos;s is the the . . . .
Attention Weights: tensor([[3.1292e-05, 3.6202e-03, 9.7324e-02, 3.3806e-01, 3.9314e-01, 1.3732e-01,
         2.6389e-02, 3.2184e-03, 5.8511e-04, 3.1986e-04],
        [1.8756e-04, 1.2437e-02, 1.4713e-01, 3.3343e-01, 3.6238e-01, 1.2223e-01,
         2.0536e-02, 1.5367e-03, 1.1547e-04, 1.8871e-05],
        [3.5653e-04, 2.0291e-02, 1.5693e-01, 2.9468e-01, 3.1801e-01, 1.5951e-01,
         4.4742e-02, 5.0739e-03, 3.6289e-04, 4.0808e-05],
        [1.1531e-03, 2.9656e-02, 1.3656e-01, 2.1912e-01, 2.4009e-01, 1.9036e-01,
         1.1862e-01, 4.6776e-02, 1.2559e-02, 5.1102e-03],
        [9.0547e-04, 1.5486e-02, 6.8923e-02, 1.1573e-01, 1.3175e-01, 1.2594e-01,
         1.1540e-01, 1.0272e-01, 1.0496e-01, 2.1819e-01],

Epoch: 0.24, Train Loss: 0.00, Val Loss: 6.09, Train BLEU: 0.00, Val BLEU: 1.22, Minutes Elapsed: 12.69
Sampling from val predictions...
Source: bức này được chụp vài tuần sau sự_kiện 11/9 ,
Reference: this one was taken just weeks after 9 /
Model: <SOS> so i , , , , , , ,
Attention Weights: tensor([[1.0457e-06, 1.5149e-05, 6.0338e-04, 2.2053e-02, 1.4643e-01, 4.2314e-01,
         3.2781e-01, 7.0959e-02, 8.1638e-03, 8.2759e-04],
        [4.2169e-06, 7.0708e-05, 2.3426e-03, 4.7131e-02, 1.8065e-01, 3.9584e-01,
         2.9627e-01, 7.0941e-02, 6.4913e-03, 2.6289e-04],
        [9.4693e-06, 1.7831e-04, 4.9891e-03, 6.2688e-02, 1.8065e-01, 3.1932e-01,
         2.8208e-01, 1.2204e-01, 2.5975e-02, 2.0665e-03],
        [1.2596e-04, 9.5308e-04, 8.7100e-03, 4.9105e-02, 1.1280e-01, 1.7860e-01,
         1.9239e-01, 1.6182e-01, 1.4628e-01, 1.4921e-01],
        [1.1593e-04, 5.2860e-04, 3.1579e-03, 1.5769e-02, 3.9052e-02, 6.7206e-02,
         8.0280e-02, 8.3599e-02, 1.4264e-01, 5.6765e-01],
        [7.1

...

Epoch: 5.53, Train Loss: 0.00, Val Loss: 4.27, Train BLEU: 0.00, Val BLEU: 13.29, Minutes Elapsed: 288.54
Sampling from val predictions...
Source: em không nói được , nhưng em truyền_đạt niềm_vui theo
Reference: he &apos;s <UNK> , but he communicates joy in
Model: <SOS> he don &apos;t saying , but the to .
Attention Weights: tensor([[9.5321e-01, 4.6575e-02, 2.0316e-04, 1.3195e-05, 2.9349e-07, 3.6517e-08,
         6.6555e-09, 1.0601e-08, 2.4377e-09, 7.0583e-10],
        [1.2465e-02, 9.0829e-01, 7.4904e-02, 4.2423e-03, 9.9313e-05, 1.8091e-06,
         3.0432e-07, 2.2013e-07, 1.9070e-08, 3.4222e-09],
        [1.8304e-02, 7.2320e-01, 1.9153e-01, 6.1535e-02, 5.1218e-03, 2.3747e-04,
         3.0816e-05, 4.0332e-05, 2.1435e-06, 3.7987e-07],
        [5.1441e-02, 1.7014e-01, 3.6884e-01, 3.7150e-01, 2.7961e-02, 8.2541e-03,
         1.0404e-03, 7.2364e-04, 8.0293e-05, 1.3577e-05],
        [2.0863e-02, 8.5155e-02, 1.3034e-01, 5.0857e-01, 1.2516e-01, 1.1723e-01,
         8.7700e-03, 3.3993e-03, 4.0

Epoch: 5.58, Train Loss: 0.00, Val Loss: 4.24, Train BLEU: 0.00, Val BLEU: 13.37, Minutes Elapsed: 291.02
Sampling from val predictions...
Source: nó là nhựa xốp , như tôi đã nói ,
Reference: it &apos;s a porous asphalt , like i said
Model: <SOS> it &apos;s a , , but i i said
Attention Weights: tensor([[9.1883e-01, 7.4521e-02, 5.8769e-03, 7.4529e-04, 2.0742e-05, 4.6568e-06,
         1.5029e-06, 4.5711e-07, 1.1703e-07, 4.6865e-08],
        [4.7247e-02, 8.6124e-01, 8.5180e-02, 6.3040e-03, 2.2297e-05, 4.2974e-06,
         9.0730e-07, 3.4296e-07, 5.2972e-08, 6.1196e-09],
        [9.3792e-03, 3.9573e-02, 6.4511e-01, 2.9835e-01, 5.8816e-03, 1.1847e-03,
         2.2513e-04, 2.3012e-04, 5.9591e-05, 4.7371e-06],
        [7.0504e-04, 2.4206e-03, 1.5258e-01, 7.9682e-01, 4.1483e-02, 5.0310e-03,
         5.8449e-04, 2.5707e-04, 9.2591e-05, 2.1827e-05],
        [1.4272e-03, 4.2949e-03, 3.0613e-02, 1.5279e-01, 2.8480e-01, 5.0272e-01,
         1.7243e-02, 3.9308e-03, 1.8997e-03, 2.7784e-04],
        [

Epoch: 5.62, Train Loss: 0.00, Val Loss: 4.25, Train BLEU: 0.00, Val BLEU: 13.81, Minutes Elapsed: 293.52
Sampling from val predictions...
Source: đọc nó từ một người phụ_nữ châu phi , những
Reference: read it from an african woman , the damage
Model: <SOS> now it from a woman , , , the
Attention Weights: tensor([[9.6350e-01, 3.5716e-02, 7.7319e-04, 6.0623e-06, 8.1880e-08, 2.1072e-08,
         5.1887e-09, 4.2386e-09, 7.6857e-10, 3.8609e-10],
        [1.7406e-01, 5.5660e-01, 2.6327e-01, 5.9624e-03, 9.2237e-05, 1.7286e-05,
         1.2465e-06, 1.2590e-07, 2.1837e-09, 5.0639e-10],
        [5.1527e-02, 8.0515e-02, 6.1391e-01, 2.1004e-01, 3.4764e-02, 8.0128e-03,
         1.1571e-03, 6.6584e-05, 3.8546e-07, 5.7777e-08],
        [1.2008e-04, 2.5617e-04, 1.9664e-02, 2.2101e-01, 4.7469e-01, 1.9660e-01,
         8.3106e-02, 4.5129e-03, 4.0033e-05, 3.7961e-06],
        [6.8195e-05, 1.2446e-04, 9.1695e-03, 8.1123e-02, 2.3135e-01, 2.9986e-01,
         3.3623e-01, 4.0223e-02, 1.7374e-03, 1.1246e-04]

Epoch: 5.67, Train Loss: 0.00, Val Loss: 4.25, Train BLEU: 0.00, Val BLEU: 13.52, Minutes Elapsed: 296.04
Sampling from val predictions...
Source: cộng_đồng của tôi , người maasai , tin rằng chúng_tôi
Reference: my community , the maasai , we believe that
Model: <SOS> my my , , , , , believe that
Attention Weights: tensor([[9.1548e-01, 7.9990e-02, 4.5197e-03, 7.3401e-06, 1.3278e-06, 2.4497e-07,
         2.7941e-08, 9.0947e-09, 7.2969e-09, 2.7765e-09],
        [5.0389e-01, 3.7047e-01, 1.2034e-01, 4.0400e-03, 9.0819e-04, 3.3526e-04,
         1.1337e-05, 1.5724e-06, 6.7872e-07, 1.9393e-07],
        [4.4981e-02, 1.0631e-01, 4.2840e-01, 2.5555e-01, 1.2866e-01, 3.3424e-02,
         2.5446e-03, 8.5172e-05, 4.5497e-05, 3.1053e-06],
        [8.6478e-04, 2.1439e-03, 8.1465e-03, 3.5293e-02, 8.5410e-01, 9.0845e-02,
         6.8660e-03, 1.4686e-03, 2.3591e-04, 4.0005e-05],
        [1.6035e-03, 6.9032e-03, 1.8706e-02, 3.4794e-02, 5.2284e-01, 3.8285e-01,
         2.5380e-02, 5.7279e-03, 1.0976e-03, 9

Epoch: 5.72, Train Loss: 0.00, Val Loss: 4.22, Train BLEU: 0.00, Val BLEU: 13.66, Minutes Elapsed: 298.55
Sampling from val predictions...
Source: trong_suốt những năm_qua đã có sự tiến_bộ to_lớn trong quá_trình
Reference: so great progress and treatment has been made over
Model: <SOS> while the have have have are a a in
Attention Weights: tensor([[9.2538e-01, 7.3918e-02, 6.5911e-04, 4.3386e-05, 1.6783e-06, 7.5514e-08,
         2.0123e-08, 5.4420e-09, 3.2313e-09, 1.1211e-09],
        [7.8446e-02, 7.9355e-01, 1.1568e-01, 1.1813e-02, 4.5259e-04, 4.9327e-05,
         7.2179e-06, 1.3396e-06, 2.7955e-07, 6.1073e-08],
        [7.0423e-03, 2.1553e-01, 4.8287e-01, 2.2814e-01, 4.0583e-02, 1.8884e-02,
         6.4439e-03, 4.9113e-04, 1.6472e-05, 2.9586e-06],
        [8.0316e-04, 4.3971e-02, 1.5783e-01, 4.9009e-01, 1.8865e-01, 9.1543e-02,
         2.3221e-02, 3.6599e-03, 1.8535e-04, 4.7784e-05],
        [4.3411e-05, 2.8524e-03, 4.6404e-02, 2.2984e-01, 2.6235e-01, 2.7231e-01,
         1.4084e-01, 

In [ ]:
experiment_results = load_experiment_log(experiment_name=EXPERIMENT_NAME)

In [ ]:
plot_single_learning_curve(experiment_results[0]['results'])

In [ ]:
summarize_results(experiment_results)[['best_val_loss', 'best_val_bleu', 'runtime', 
                                       'total_params', 'trainable_params', 'dt_created']]

In [ ]:
# reload model and test 
checkpoint = torch.load('model_checkpoints/{}.pth.tar'.format(MODEL_NAME), map_location=device)
model.load_state_dict(checkpoint)